In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 加载数据

In [ ]:
from fastai.vision import *
from pathlib import Path

In [ ]:
root = Path("../input")
root
root.as_posix()

- 读入标签

In [ ]:
train_df = pd.read_csv(root/"train.csv")
test_df = pd.read_csv(root/"sample_submission.csv")

In [ ]:
train_df.head()
test_df.head()

- 测试集构造

In [ ]:
test_set = ImageList.from_df(test_df, path=root/'test', cols='id', folder='test')

In [ ]:
test_set

- transforms 定义

In [ ]:
tsfm = get_transforms(do_flip=True, flip_vert=True, max_rotate=10.0, max_zoom=1.1, max_lighting=0.2, max_warp=0.2, p_affine=0.75, p_lighting=0.75)

- databunch 构造

In [ ]:
SZ=128
BS=64

In [ ]:
np.random.seed(42)
data = (ImageList.from_df(train_df, path=root/'train', cols='id', folder='train')
       .split_by_rand_pct(0.01)
       .label_from_df()
       .transform(tsfm, size=SZ)
        .add_test(test_set)
       .databunch(path='./', bs=BS, device= torch.device('cuda:0'))
       .normalize(imagenet_stats)
      )

In [ ]:
data

In [ ]:
data.show_batch(rows=3, figsize=(6,6))

- 构造模型

## 使用 Densenet

In [ ]:
# arch = models.densenet161
arch = models.densenet169
# arch = models.densenet121
# arch = resnet50 # 也可以达到 1

In [ ]:
learn = cnn_learner(data, arch, metrics=[error_rate, accuracy])

- 寻找最佳学习率

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-03
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.recorder.plot_losses()

- 第二阶段
由于第一阶段已经达到 1 所以我们可以不进行第二阶段的训练

In [ ]:
# learn.unfreeze()

- 再次寻找学习率

In [ ]:
# learn.lr_find(1e-10,10)
# learn.recorder.plot(skip_end=15)

In [ ]:
# lr1 = 5e-6
# learn.fit_one_cycle(2, slice(lr1/2.6**3, lr1))

In [ ]:
# data1 = (ImageList.from_df(train_df, path=root/'train', cols='id', folder='train')
#        .split_by_rand_pct(0.01)
#        .label_from_df()
#        .transform(tsfm, size=224)
#         .add_test(test_set)
#        .databunch(path='./', bs=64, device= torch.device('cuda:0'))
#        .normalize(imagenet_stats)
#       )

In [ ]:
# learn.data = data1

In [ ]:
# learn.freeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# lr2 = 1e-3
# learn.fit_one_cycle(3, lr2)

In [ ]:
# learn.unfreeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# lr3 = 1e-5
# learn.fit_one_cycle(2, slice(lr3/2.6**3,lr3))

In [ ]:
learn.recorder.plot_losses()

## 预测并提交结果

In [ ]:
preds,_ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
test_df.has_cactus = preds.numpy()[:, 0]

In [ ]:
test_df.to_csv('submission.csv', index=False)

- 提交结果